In [1]:
import pandas as pd
import re
import string
import nltk
pd.set_option('display.max_colwidth', 100)
import glob
stopwords = nltk.corpus.stopwords.words('english')

In [2]:
wn = nltk.WordNetLemmatizer()

In [5]:
path ='/home/ubuntu/nltk_data/corpora/movie_reviews/neg' # use your path
allFiles = glob.glob(path + "/*.txt")
frame = pd.DataFrame()
list_ = []
for file_ in allFiles:
    df = pd.read_table(file_, header=None)
    list_.append(df)
frame = pd.concat(list_)

In [6]:
frame['label']=0

In [5]:
frame=frame.head(5000)

In [7]:
path =r'/home/ubuntu/nltk_data/corpora/movie_reviews/pos' # use your path
allFiles = glob.glob(path + "/*.txt")
frame2 = pd.DataFrame()
list_ = []
for file_ in allFiles:
    df = pd.read_table(file_, header=None)
    list_.append(df)
frame2 = pd.concat(list_)

In [8]:
frame2['label']=1

In [8]:
frame2=frame2.head(5000)

In [9]:
final_data=frame.append(frame2)

In [10]:
final_data.shape

(64720, 2)

In [ ]:
def remove_punct(text):
    text_nopunct = "".join([char for char in text if char not in string.punctuation])
    return text_nopunct

final_data['body_text_clean'] = final_data[0].apply(lambda x: remove_punct(x))

final_data.head()

In [14]:
final_data.rename(columns={0:'text'},inplace=True)

In [21]:

def clean_text(text):
    text = "".join([word for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = [wn.lemmatize(word) for word in tokens if word not in stopwords]
    return text

In [15]:
s=clean_text(final_data['text'])

In [15]:
final_data.tail()

,text,label
19,"when true crime opts for a high-speed chase to the governer's house at the finale , the quality ...",1
20,eastwood was so successful with colorful character portraits that he didn't need to switch lanes .,1
21,"true crime is a tension-building , intriguing drama showcase for the talented director and star .",1
22,this is a road block he could have easily dismissed ( i furrow my brow ) .,1
23,. . . . . . . . . . . . . . . . . . . . . . . . . . .,1


In [16]:
def standardize_text(df, text_field):
    df[text_field] = df[text_field].str.replace(r"http\S+", "")
    df[text_field] = df[text_field].str.replace(r"http", "")
    df[text_field] = df[text_field].str.replace(r"@\S+", "")
    df[text_field] = df[text_field].str.replace(r"[^A-Za-z0-9(),!?@\'\`\"\_\n]", " ")
    df[text_field] = df[text_field].str.replace(r"@", "at")
    df[text_field] = df[text_field].str.lower()
    df[text_field] = df[text_field].str.replace('\s+', ' ')
    df[text_field] = df[text_field].str.replace("\'", "")
    return df

In [17]:
final_data = standardize_text(final_data, "text")

In [14]:
final_data.tail()

,text,label
56,"it's a quick , straight shot to the movie's end",1
57,"in terms of overall quality , i would compare the truman show to niccol's gattaca",1
58,both films are well made with interesting stories set in interesting worlds,1
59,"but neither film really felt like it capitalized on all the great ideas neither film "" clicked...",1
60,"nevertheless , i look forward to niccol's next film , whatever it may be",1


In [18]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(final_data[['text']], final_data['label'], test_size=0.2)


In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer,HashingVectorizer

In [22]:
tfidf_vect = TfidfVectorizer(analyzer=clean_text)
tfidf_vect_fit = tfidf_vect.fit(X_train['text'])
tfidf_train = tfidf_vect_fit.transform(X_train['text'])
tfidf_test = tfidf_vect_fit.transform(X_test['text'])

In [23]:
# hv=HashingVectorizer(analyzer=clean_text,n_features=25000 ,non_negative=True,ngram_range=(2,2))
# x_train_hv=hv.transform(X_train['text'])
# x_train_hv_vec=pd.DataFrame(x_train_hv.toarray())
# x_test_hv=hv.transform(X_test['text'])
# x_test_hv_vec=pd.DataFrame(x_test_hv.toarray())

In [ ]:
# X_train_vect =pd.DataFrame(tfidf_train.toarray())
# X_test_vect =pd.DataFrame(tfidf_test.toarray())

In [20]:
y_test.shape

(2000,)

In [26]:
from sklearn.ensemble import RandomForestClassifier,BaggingClassifier
from sklearn.naive_bayes import MultinomialNB
classifier = MultinomialNB()

In [27]:
classifier.fit(tfidf_train, y_train)

y_pred = classifier.predict(tfidf_test)
#y_real_pred = classifier.predict(X_test_vect)

from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

             precision    recall  f1-score   support

          0       0.71      0.68      0.69      6365
          1       0.70      0.73      0.71      6579

avg / total       0.70      0.70      0.70     12944



In [28]:
rf=RandomForestClassifier(n_estimators=150, max_depth=3, n_jobs=-1)
model=rf.fit(tfidf_train, y_train)
pred=model.predict(tfidf_test)
print(classification_report(y_test, pred))

             precision    recall  f1-score   support

          0       0.65      0.62      0.64      6365
          1       0.65      0.68      0.66      6579

avg / total       0.65      0.65      0.65     12944



In [29]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

In [31]:
# rf = RandomForestClassifier()
# param = {'n_estimators': [10, 150, 300],
#         'max_depth': [30, 60, 90, None]}

# gs = GridSearchCV(rf, param, cv=5, n_jobs=-1)
# gs_fit = gs.fit(tfidf_train, y_train)
# pd.DataFrame(gs_fit.cv_results_).sort_values('mean_test_score', ascending=False)[0:5]

In [38]:
tfidf_vect = TfidfVectorizer(analyzer=clean_text,ngram_range=(2,2))
tfidf_vect_fit = tfidf_vect.fit(X_train['text'])
tfidf_train = tfidf_vect_fit.transform(X_train['text'])
tfidf_test = tfidf_vect_fit.transform(X_test['text'])

In [39]:
classifier.fit(tfidf_train, y_train)

y_pred = classifier.predict(tfidf_test)
#y_real_pred = classifier.predict(X_test_vect)

from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

             precision    recall  f1-score   support

          0       0.71      0.68      0.69      6365
          1       0.70      0.73      0.71      6579

avg / total       0.70      0.70      0.70     12944



In [37]:
rf=RandomForestClassifier(n_estimators=150, max_depth=3, n_jobs=-1)
model=rf.fit(tfidf_train, y_train)
pred=model.predict(tfidf_test)
print(classification_report(y_test, pred))

             precision    recall  f1-score   support

          0       0.82      0.04      0.08      6365
          1       0.52      0.99      0.68      6579

avg / total       0.67      0.52      0.38     12944



In [40]:
from sklearn.linear_model import LogisticRegression
clf_tfidf = LogisticRegression(C=30.0, class_weight='balanced', solver='newton-cg', n_jobs=-1, random_state=40)

In [41]:
model=clf_tfidf.fit(tfidf_train, y_train)
pred=model.predict(tfidf_test)
print(classification_report(y_test, pred))

             precision    recall  f1-score   support

          0       0.68      0.68      0.68      6365
          1       0.69      0.68      0.69      6579

avg / total       0.68      0.68      0.68     12944



In [97]:
reviews_new = ['This movie was excellent', 'Absolute joy ride', 
            'Adam was terrible', 'Meryl shined through.', 'wasted my money','i was better at home','rocking!!',
              'This was certainly a movie', 'Two thumbs up', 'I fell asleep halfway through', 'first half was good..second half was boring',
              "We can't wait for the sequel!!", '!', '?', 'I cannot recommend this highly enough', ':-D',':-(',';-(',':-X',
              'instant classic.', 'Brad Pitt was amazing. His performance was Oscar-worthy.','this movie is so bad, i ran away','amazing']

#hash_vect_sample = TfidfVectorizer(analyzer=clean_text ,ngram_range=(2,2))
test = tfidf_vect_fit.transform(reviews_new)

In [102]:
test_pred=model.predict(test)

In [103]:
(test_pred)=pd.DataFrame(test_pred)

In [104]:
test_pred.rename(columns={0:'label'},inplace=True)
test_pred['label']=np.where(test_pred['label']==0,'neg','pos')

In [101]:
# naive bayes
for review, category in zip(reviews_new, test_pred['label']):
    print('%r => %s' % (review, category))

'This movie was excellent' => pos
'Absolute joy ride' => pos
'Adam was terrible' => neg
'Meryl shined through.' => pos
'wasted my money' => neg
'i was better at home' => neg
'rocking!!' => pos
'This was certainly a movie' => neg
'Two thumbs up' => neg
'I fell asleep halfway through' => neg
'first half was good..second half was boring' => neg
"We can't wait for the sequel!!" => neg
'!' => neg
'?' => neg
'I cannot recommend this highly enough' => pos
':-D' => pos
':-(' => neg
';-(' => neg
':-X' => pos
'instant classic.' => pos
'Brad Pitt was amazing. His performance was Oscar-worthy.' => pos
'this movie is so bad, i ran away' => neg
'amazing' => pos


In [105]:
# logistic regression
for review, category in zip(reviews_new, test_pred['label']):
    print('%r => %s' % (review, category))

'This movie was excellent' => pos
'Absolute joy ride' => pos
'Adam was terrible' => neg
'Meryl shined through.' => neg
'wasted my money' => neg
'i was better at home' => neg
'rocking!!' => pos
'This was certainly a movie' => neg
'Two thumbs up' => neg
'I fell asleep halfway through' => neg
'first half was good..second half was boring' => neg
"We can't wait for the sequel!!" => neg
'!' => neg
'?' => neg
'I cannot recommend this highly enough' => pos
':-D' => neg
':-(' => neg
';-(' => neg
':-X' => neg
'instant classic.' => pos
'Brad Pitt was amazing. His performance was Oscar-worthy.' => pos
'this movie is so bad, i ran away' => neg
'amazing' => pos


In [89]:
import xgboost as xgb
from xgboost.sklearn import XGBClassifier

In [91]:
# cv_params = {'max_depth': np.arange(3,12,2), 'min_child_weight': [1,3,5]}
# ind_params = {'learning_rate': 0.1, 'n_estimators': 1000, 'seed':0, 'subsample': 0.8, 'colsample_bytree': 0.8, 
#              'objective': 'binary:logistic'}
# optimized_GBM = GridSearchCV(xgb.XGBClassifier(**ind_params), 
#                             cv_params, 
#                              scoring = 'accuracy', cv = 5, n_jobs = -1) 
# # Optimize for accuracy since that is the metric used in the Adult Data Set notation
# optimized_GBM.fit(tfidf_train, y_train)

In [ ]:
optimized_GBM.grid_scores_

In [ ]:
from sklearn.model_selection import RandomizedSearchCV